In [ ]:
from qiskit import ClassicalRegister
from qiskit import QuantumCircuit
from qiskit import QuantumRegister
from qiskit.circuit.library import MCXGate
from qiskit_aer import AerSimulator

from vbe_arithmetic import ripple_add_modulo_n, set_reg_bits

In [ ]:
# !!!!! IMPORTANT !!!!!
# This implementation has not been tested

x_dec = 2
x_len = x_dec.bit_length()

a_dec = 6

n_dec = 7

# 2^i * a
# Used for modular addition below
a_mult = [(2 ** i) * a_dec for i in range(0, x_len)]
a_len = a_mult[-1].bit_length()

# Get the max bit length between all of the above numbers
# Will be used to create registers of this size
N = max(x_len, n_dec.bit_length())
N = max(N, a_len)

ctrl = QuantumRegister(1, 'ctrl')  # Control bit for multiplication
n = QuantumRegister(N, 'mod')  # Register to store modulo value
a = QuantumRegister(a_len, 'a')
x = QuantumRegister(N, 'x')  # Register for exponential
t = QuantumRegister(N, 'temp')  # Temp register, first operand to adder-mod
r = QuantumRegister(N * 2, 'result')  # Result register, second operand to adder-mod
adder_t = QuantumRegister(N, 'adder_t')  # Temp register used in adder-mod
c = QuantumRegister(N, 'carry')  # Carry register used in adder-mod
of = QuantumRegister(1, 'of')  # Overflow bit, used in adder-mod
cx = ClassicalRegister(N, 'cx')
cr = ClassicalRegister(N * 2, 'cr')

qc = QuantumCircuit(ctrl, n, a, x, r, t, c, of, adder_t, cx, cr)

# Set the modulo bits
set_reg_bits(qc, n_dec, n)

# Set the control qubit to |1> state
qc.x(ctrl[0])

for i in range(0, x_len):
    # Copy (2^i * a) into the temporary register if control qbit and x_i is in state |1>
    # We use control-control-control-NOT gate to copy the bits
    # The first two controls are used to trigger the copy operation
    # Then the last control-NOT copies the bits

    for j in range(0, a_len):

        # Copy the current (2^i * a) into the 'a' register
        if a_mult[i] & (1 << j) != 0:
            qc.x(a[j])

        # if a[j] is in |1> state, copy it to the temporary register
        mcx = MCXGate(3)
        qc.append(mcx, [ctrl[0], x[i], a[j], t[j]])

    # At this point, the temporary register contains the bits from a_mult[i]
    # Apply the adder-mod operation, with :
    # - temporary register as first operand
    # - result register as the second operand
    ripple_add_modulo_n(qc, N, t, r, c, n, of, adder_t)

    # Reset the value in the 'a' register
    for j in range(0, a_len):
        qc.reset(a[j])


In [ ]:
#qc.draw(output="mpl", filename="circuits/1996_adder.png", style="clifford")

In [ ]:
qc.measure(x, cx)
qc.measure(r, cr)

backend_sim = AerSimulator()
job_sim = backend_sim.run(qc, shots=1)
result_sim = job_sim.result()

print(result_sim.get_counts(qc))